In [1]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 

import numpy as np
import pandas as pd
import re

import joblib
from tqdm.auto import tqdm
tqdm.pandas()

from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import model_selection, dummy 
from sklearn.metrics import accuracy_score, confusion_matrix,f1_score
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.svm import LinearSVC

import nltk
from nltk.corpus import stopwords
from nltk import sent_tokenize



/home/unicph.domain/xwf576/anaconda3/lib/python3.7/site-packages/sklearn/feature_extraction/image.py:167: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dtype=np.int):
/home/unicph.domain/xwf576/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/least_angle.py:35: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; fo

In [2]:
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     /home/unicph.domain/xwf576/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/unicph.domain/xwf576/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
#set seed
np.random.seed(1)
#tqdm
tqdm.pandas()
# read data
df_label = pd.read_parquet("data/code_sagsomkostninger.parquet")
danish_stopwords = stopwords.words('danish')
#remove more data
df_label = df_label.loc[~df_label["LABEL_party_who_pays"].isna()]
df_ufr = pd.read_parquet("data/UfR_text.parquet")


In [4]:
df = pd.merge(df_label.loc[:,["id_verdict","LABEL_party_who_pays","omkostninger"]],df_ufr)

In [5]:
def mask_omkostnigner(df):
    sentences_wo_omkostninger = [sentence for sentence in sent_tokenize(df["verdict_text"], language="danish") if sentence not in df["omkostninger"]]
    return " ".join(sentences_wo_omkostninger)

df["verdict_text_masked"] = df.progress_apply(mask_omkostnigner,axis=1)

  0%|          | 0/13876 [00:00<?, ?it/s]

### Create 80-20 train test split - masked

In [6]:
Train_X, Test_X, Train_Y, Test_Y = model_selection.train_test_split(df['verdict_text'],df['LABEL_party_who_pays'],test_size=0.20,random_state=1)
Encoder = LabelEncoder()
Train_Y = Encoder.fit_transform(Train_Y)
Test_Y = Encoder.fit_transform(Test_Y)
Train_X_masked = df.loc[Train_X.index,"verdict_text_masked"] 
Test_X_masked = df.loc[Test_X.index,"verdict_text_masked"] 
train_ids = df.loc[Train_X.index,"id_verdict"]
test_ids = df.loc[Test_X.index,"id_verdict"]
train_ids.to_csv("document_ids_train")
test_ids.to_csv("document_ids_test")


### Train Linear SVC

#### Set parameters

In [7]:

pipe = Pipeline([("tfidf",TfidfVectorizer(analyzer='word')   ), ("svc", LinearSVC())])
param_grid = {
    'tfidf__ngram_range': [(1,1),(1,2),(1,4),(2,2),(2,4)],
	'tfidf__lowercase': [True],
	'tfidf__min_df': (1, 2, 3), # we need to see a word at least (once, twice)
	'tfidf__use_idf': [True],
	'tfidf__binary': [True],
	'tfidf__norm': ['l2'],
	'tfidf__stop_words': (None, danish_stopwords),
    'svc__C':(0.1, 0.5, 1, 5)
}

#### Train masked

In [ ]:
if __name__=="__main__":
    search = GridSearchCV(pipe,param_grid, verbose = 2, cv=5, n_jobs=-1)
    search.fit(Train_X_masked, Train_Y)
    #save model 
    print("saving model")
    joblib.dump(search, 'results/grid_search_object_masked.pkl')

#### Train not masked

In [ ]:
search = GridSearchCV(pipe,param_grid, verbose = 2, cv=5, n_jobs=-1)
search.fit(Train_X, Train_Y)
#save model 
print("saving model")
joblib.dump(search, 'results/grid_search_object_not_masked.pkl')

## Evaluation

#### Load data

In [9]:
masked = joblib.load('results/grid_search_object_masked.pkl')

In [10]:
not_masked = joblib.load('results/grid_search_object_not_masked.pkl')

### Metrics for each classifier

In [11]:
def print_scores(predictions, targets):
    print("SVM Accuracy Score ->",accuracy_score(predictions, targets)*100)
    print("F1 Score ->",f1_score(predictions, targets,average="macro")*100)
    print("Confusion matrix: \n",confusion_matrix(predictions, targets))

#### Predictions masked using masked 

In [12]:
# Classifier - Algorithm - SVM
# fit the training dataset on the classifier
# predict the labels on validation dataset
predictions_SVM_masked = masked.predict(Test_X_masked)
# Use accuracy_score function to get the accuracy
print_scores(predictions_SVM_masked,Test_Y)


SVM Accuracy Score -> 69.05619596541787
F1 Score -> 61.77617166540678
Confusion matrix: 
 [[ 588  124  186]
 [  32  145   41]
 [ 293  183 1184]]


In [44]:
masked.best_estimator_.steps

[('tfidf', TfidfVectorizer(analyzer='word', binary=True, decode_error='strict',
          dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
          lowercase=True, max_df=1.0, max_features=None, min_df=3,
          ngram_range=(2, 4), norm='l2', preprocessor=None, smooth_idf=True,
          stop_words=['og', 'i', 'jeg', 'det', 'at', 'en', 'den', 'til', 'er', 'som', 'på', 'de', 'med', 'han', 'af', 'for', 'ikke', 'der', 'var', 'mig', 'sig', 'men', 'et', 'har', 'om', 'vi', 'min', 'havde', 'ham', 'hun', 'nu', 'over', 'da', 'fra', 'du', 'ud', 'sin', 'dem', 'os', 'op', 'man', 'hans', 'hvor',..., 'din', 'nogle', 'hos', 'blive', 'mange', 'ad', 'bliver', 'hendes', 'været', 'thi', 'jer', 'sådan'],
          strip_accents=None, sublinear_tf=False,
          token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
          vocabulary=None)),
 ('svc', LinearSVC(C=5, class_weight=None, dual=True, fit_intercept=True,
       intercept_scaling=1, loss='squared_hinge', max_iter=

#### Predictions not-masked using not-masked

In [14]:
# Classifier - Algorithm - SVM
# fit the training dataset on the classifier
# predict the labels on validation dataset
predictions_SVM_not_masked = not_masked.predict(Test_X)
# Use accuracy_score function to get the accuracy
print_scores(predictions_SVM_not_masked,Test_Y)


SVM Accuracy Score -> 87.71613832853026
F1 Score -> 87.34881329308882
Confusion matrix: 
 [[ 758   39   93]
 [   6  367    8]
 [ 149   46 1310]]


In [43]:
not_masked.best_estimator_.steps

[('tfidf', TfidfVectorizer(analyzer='word', binary=True, decode_error='strict',
          dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
          lowercase=True, max_df=1.0, max_features=None, min_df=3,
          ngram_range=(2, 4), norm='l2', preprocessor=None, smooth_idf=True,
          stop_words=['og', 'i', 'jeg', 'det', 'at', 'en', 'den', 'til', 'er', 'som', 'på', 'de', 'med', 'han', 'af', 'for', 'ikke', 'der', 'var', 'mig', 'sig', 'men', 'et', 'har', 'om', 'vi', 'min', 'havde', 'ham', 'hun', 'nu', 'over', 'da', 'fra', 'du', 'ud', 'sin', 'dem', 'os', 'op', 'man', 'hans', 'hvor',..., 'din', 'nogle', 'hos', 'blive', 'mange', 'ad', 'bliver', 'hendes', 'været', 'thi', 'jer', 'sådan'],
          strip_accents=None, sublinear_tf=False,
          token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
          vocabulary=None)),
 ('svc', LinearSVC(C=5, class_weight=None, dual=True, fit_intercept=True,
       intercept_scaling=1, loss='squared_hinge', max_iter=

#### Prediction not masked model using masked data

In [16]:
# Classifier - Algorithm - SVM
# fit the training dataset on the classifier
# predict the labels on validation dataset
predictions_SVM_not_masked_on_masked = not_masked.predict(Test_X_masked)
# Use accuracy_score function to get the accuracy
print_scores(predictions_SVM_not_masked_on_masked,Test_Y)


SVM Accuracy Score -> 66.1743515850144
F1 Score -> 59.43333294367831
Confusion matrix: 
 [[ 542  114  211]
 [  47  151   56]
 [ 324  187 1144]]


#### Dummy classifier

In [17]:
dummy_classifier = dummy.DummyClassifier()
Tfidf_vect = TfidfVectorizer(analyzer='word')
Tfidf_vect.fit(df["verdict_text"])
Train_X_Tfidf = Tfidf_vect.transform(Train_X)
Test_X_Tfidf = Tfidf_vect.transform(Test_X)
dummy_classifier.fit(Train_X_Tfidf,Train_Y)
predictions_DUMMY = dummy_classifier.predict(Test_X_Tfidf)
print_scores(predictions_DUMMY,Test_Y)

SVM Accuracy Score -> 38.79682997118156
F1 Score -> 33.73792424614513
Confusion matrix: 
 [[294 143 468]
 [156  91 251]
 [463 218 692]]


/home/unicph.domain/xwf576/anaconda3/lib/python3.7/site-packages/sklearn/dummy.py:227: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  k in range(self.n_outputs_)).T


### Evaluate weights

In [18]:
def print_top10(vectorizer, clf, class_labels):
    """Prints features with the highest coefficient values, per class"""
    feature_names = vectorizer.get_feature_names()
    for i, class_label in enumerate(class_labels):
        top10 = np.argsort(clf.coef_[i])[-30:]
        print("%s: %s" % (class_label,
              ", ".join(feature_names[j] for j in top10)))
def print_bot10(vectorizer, clf, class_labels):
    """Prints features with the highest coefficient values, per class"""
    feature_names = vectorizer.get_feature_names()
    for i, class_label in enumerate(class_labels):
        top10 = np.argsort(clf.coef_[i])[:30]
        print("%s: %s" % (class_label,
              ", ".join(feature_names[j] for j in top10)))

#### Print the absolute largest coefficient for the masked classifier:

In [19]:
tf_idf = masked.best_estimator_.get_params()["steps"][0][1]
tf_idf.fit(df["verdict_text_masked"])
svc = masked.best_estimator_.get_params()["steps"][1][1]
tf_idf_X_Train = tf_idf.transform(Train_X_masked)
svc.fit(tf_idf_X_Train,Train_Y)
print_top10(tf_idf,svc,svc.classes_)

0: appellantens påstand tage følge, appellantens påstand tage, sagen stadfæstes, dom sagen stadfæstes, findes passende, findes indstævnte, tiltale indstævnte, stemmeflertallet kendes ret indstævnte, tage følge, sagsøgerens påstand, bør tiltale indstævnte, tager herefter, bør appellanten, højesteret tager herefter, ret appellanten, kendes ret appellanten, appellanternes påstand, følge kendes ret indstævnte, følge beløbet, tager derfor, påstand tage, påstand tage følge, kr retsafgift, påstand følge, følge kendes ret, følge kendes, højesteret tager, appellantens påstand, ret indstævnte, kendes ret indstævnte
1: forklaring parterne, frifandtes sagsøgte, tilladelse indanket højesteret pådømmelsen, højesteret tillægges, landsret sag, indgik ægteskab, kst adv, indanket højesteret pådømmelsen, mellem ægtefællerne, parterne betaler, part bærer, hver part bærer, ved bevilling, tage appellantens, tillægges højesteretssagfører, note salær, note note salær, meddelt fri, meddelt fri proces, dommen h

#### Print the absolute largest coefficient for the not-masked classifier:

In [20]:
tf_idf = not_masked.best_estimator_.get_params()["steps"][0][1]
tf_idf.fit(df["verdict_text"])
svc = masked.best_estimator_.get_params()["steps"][1][1]
tf_idf_X_Train = tf_idf.transform(Train_X)
svc.fit(tf_idf_X_Train,Train_Y)
print_top10(tf_idf,svc,svc.classes_)

0: frifindes sagsomkostninger, stadfæstes tiltalte, højesteret udredes tiltalte, begge retter betaler indstævnte, afsigelse betale, pålagdes sagsøgte, dage højesteretsdoms afsigelse betale, højesteretsdoms afsigelse betale, indstævnte appellanten, omkostninger højesteret betales tiltalte, betaler indstævnte appellanten, højesteret betales tiltalte, kr retsafgift, findes indstævnte, sagsomkostninger landsret højesteret, sagsomkostninger landsret, retter betaler indstævnte, betales tiltalte, ret indstævnte, kendes ret indstævnte, kr udredes tiltalte, højesteret betaler indstævnte, begge retter, sagsomkostninger begge retter, sagsomkostninger begge, 000 kr appellanten, indstævnte betale, kr appellanten, udredes tiltalte, betaler indstævnte
1: landsretten part, stadfæstes ingen parterne, landsret højesteret part, stadfæstes ingen, betale sagsomkostninger part, burde ophæves, part statskassen, sagsomkostninger højesteret part, burde bære, burde bære sagsomkostninger, betale sagsomkostninger

In [21]:
print_bot10(tf_idf,svc,svc.classes_)

0: statskassen betale sagens omkostninger, statskassen betale sagens, statskassen betale, udredes offentlige, dom stadfæstes sagsomkostninger, ingen parterne, stadfæstes sagsomkostninger, sagsomkostninger højesteret, højesteret betales statskassen, kr indstævnte, dom stadfæstes, betaler appellanten, betales statskassen, stadfæstes sagsomkostninger højesteret, kr udredes offentlige, dom stadfæstes sagsomkostninger højesteret, landsrettens dom stadfæstes sagsomkostninger, omkostninger højesteret betales statskassen, ret landsrettens, kendes ret landsrettens, kendes ret landsrettens dom, ret landsrettens dom, 000 kr indstævnte, statskassen note, statskassen note note, sagsomkostninger landsretten, hver part, parterne betaler, parterne betaler sagsomkostninger, ingen parterne betaler
1: dage højesteretsdoms, dage højesteretsdoms afsigelse, højesteretsdoms afsigelse, højesteret betaler, 14 dage højesteretsdoms, inden 14 dage højesteretsdoms, 14 dage højesteretsdoms afsigelse, betaler indstæ

## Criminal and civil cases

In [22]:
def straffesag(s):
    if re.search(("rigsadvokat|anklagemyndighed"),s,re.IGNORECASE):
        return True
    else:
        return False

df_label["straffesag"] = df_label["prosecutor"].apply(straffesag)
df = pd.merge(df,df_label.loc[:,["id_verdict","straffesag"]], left_on = "id_verdict", right_on = "id_verdict")


### Train criminal cases

#### Train-test split

In [23]:
df_criminal = df.loc[df["straffesag"] == True]
df_criminal = df_criminal.loc[df["LABEL_party_who_pays"]!="NO_COST"]
Train_X, Test_X, Train_Y, Test_Y = model_selection.train_test_split(df_criminal["verdict_text_masked"],df_criminal['LABEL_party_who_pays'],test_size=0.2,random_state=1)
Encoder = LabelEncoder()
Train_Y = Encoder.fit_transform(Train_Y)
Test_Y = Encoder.fit_transform(Test_Y)
train_ids = df_criminal.loc[Train_X.index,"id_verdict"]
test_ids = df_criminal.loc[Test_X.index,"id_verdict"]
train_ids.to_csv("document_ids_criminal_train")
test_ids.to_csv("document_ids_criminal_test")

#### Train 

In [ ]:
search = GridSearchCV(pipe,param_grid, verbose = 2, cv=5, n_jobs=-1)
search.fit(Train_X, Train_Y)
#save model 
print("saving model")
joblib.dump(search, 'results/grid_search_object_masked_criminal.pkl')

#### Train not-criminal cases

In [25]:
df_not_criminal = df.loc[df["straffesag"] == False]
Train_X, Test_X, Train_Y, Test_Y = model_selection.train_test_split(df_not_criminal["verdict_text_masked"],df_not_criminal['LABEL_party_who_pays'],test_size=0.2,random_state=1)
Encoder = LabelEncoder()
Train_Y = Encoder.fit_transform(Train_Y)
Test_Y = Encoder.fit_transform(Test_Y)
train_ids = df_not_criminal.loc[Train_X.index,"id_verdict"]
test_ids = df_not_criminal.loc[Test_X.index,"id_verdict"]
train_ids.to_csv("document_ids_not_criminal_train")
test_ids.to_csv("document_ids_not_criminal_test")

In [26]:
search = GridSearchCV(pipe,param_grid, verbose = 2, cv=5, n_jobs=-1)
search.fit(Train_X, Train_Y)
#save model 
print("saving model")
joblib.dump(search, 'results/grid_search_object_masked_not_criminal.pkl')

Fitting 5 folds for each of 120 candidates, totalling 600 fits


/home/unicph.domain/xwf576/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:618: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  test_folds = np.zeros(n_samples, dtype=np.int)
/home/unicph.domain/xwf576/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:428: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to 

[CV] svc__C=0.1, tfidf__binary=True, tfidf__lowercase=True, tfidf__min_df=1, tfidf__ngram_range=(1, 1), tfidf__norm=l2, tfidf__stop_words=None, tfidf__use_idf=True 
[CV] svc__C=0.1, tfidf__binary=True, tfidf__lowercase=True, tfidf__min_df=1, tfidf__ngram_range=(1, 1), tfidf__norm=l2, tfidf__stop_words=None, tfidf__use_idf=True 
[CV] svc__C=0.1, tfidf__binary=True, tfidf__lowercase=True, tfidf__min_df=1, tfidf__ngram_range=(1, 1), tfidf__norm=l2, tfidf__stop_words=None, tfidf__use_idf=True 
[CV] svc__C=0.1, tfidf__binary=True, tfidf__lowercase=True, tfidf__min_df=1, tfidf__ngram_range=(1, 1), tfidf__norm=l2, tfidf__stop_words=None, tfidf__use_idf=True 
[CV] svc__C=0.1, tfidf__binary=True, tfidf__lowercase=True, tfidf__min_df=1, tfidf__ngram_range=(1, 1), tfidf__norm=l2, tfidf__stop_words=None, tfidf__use_idf=True 
[CV] svc__C=0.1, tfidf__binary=True, tfidf__lowercase=True, tfidf__min_df=1, tfidf__ngram_range=(1, 1), tfidf__norm=l2, tfidf__stop_words=['og', 'i', 'jeg', 'det', 'at', 'en',

[CV] svc__C=0.1, tfidf__binary=True, tfidf__lowercase=True, tfidf__min_df=1, tfidf__ngram_range=(1, 2), tfidf__norm=l2, tfidf__stop_words=['og', 'i', 'jeg', 'det', 'at', 'en', 'den', 'til', 'er', 'som', 'på', 'de', 'med', 'han', 'af', 'for', 'ikke', 'der', 'var', 'mig', 'sig', 'men', 'et', 'har', 'om', 'vi', 'min', 'havde', 'ham', 'hun', 'nu', 'over', 'da', 'fra', 'du', 'ud', 'sin', 'dem', 'os', 'op', 'man', 'hans', 'hvor', 'eller', 'hvad', 'skal', 'selv', 'her', 'alle', 'vil', 'blev', 'kunne', 'ind', 'når', 'være', 'dog', 'noget', 'ville', 'jo', 'deres', 'efter', 'ned', 'skulle', 'denne', 'end', 'dette', 'mit', 'også', 'under', 'have', 'dig', 'anden', 'hende', 'mine', 'alt', 'meget', 'sit', 'sine', 'vor', 'mod', 'disse', 'hvis', 'din', 'nogle', 'hos', 'blive', 'mange', 'ad', 'bliver', 'hendes', 'været', 'thi', 'jer', 'sådan'], tfidf__use_idf=True 
[CV] svc__C=0.1, tfidf__binary=True, tfidf__lowercase=True, tfidf__min_df=1, tfidf__ngram_range=(1, 2), tfidf__norm=l2, tfidf__stop_words=[

[CV] svc__C=0.1, tfidf__binary=True, tfidf__lowercase=True, tfidf__min_df=1, tfidf__ngram_range=(2, 2), tfidf__norm=l2, tfidf__stop_words=['og', 'i', 'jeg', 'det', 'at', 'en', 'den', 'til', 'er', 'som', 'på', 'de', 'med', 'han', 'af', 'for', 'ikke', 'der', 'var', 'mig', 'sig', 'men', 'et', 'har', 'om', 'vi', 'min', 'havde', 'ham', 'hun', 'nu', 'over', 'da', 'fra', 'du', 'ud', 'sin', 'dem', 'os', 'op', 'man', 'hans', 'hvor', 'eller', 'hvad', 'skal', 'selv', 'her', 'alle', 'vil', 'blev', 'kunne', 'ind', 'når', 'være', 'dog', 'noget', 'ville', 'jo', 'deres', 'efter', 'ned', 'skulle', 'denne', 'end', 'dette', 'mit', 'også', 'under', 'have', 'dig', 'anden', 'hende', 'mine', 'alt', 'meget', 'sit', 'sine', 'vor', 'mod', 'disse', 'hvis', 'din', 'nogle', 'hos', 'blive', 'mange', 'ad', 'bliver', 'hendes', 'været', 'thi', 'jer', 'sådan'], tfidf__use_idf=True 
[CV] svc__C=0.1, tfidf__binary=True, tfidf__lowercase=True, tfidf__min_df=1, tfidf__ngram_range=(2, 2), tfidf__norm=l2, tfidf__stop_words=[

...

[CV]  svc__C=5, tfidf__binary=True, tfidf__lowercase=True, tfidf__min_df=1, tfidf__ngram_range=(1, 4), tfidf__norm=l2, tfidf__stop_words=None, tfidf__use_idf=True, total=33.4min
[CV]  svc__C=5, tfidf__binary=True, tfidf__lowercase=True, tfidf__min_df=2, tfidf__ngram_range=(2, 4), tfidf__norm=l2, tfidf__stop_words=None, tfidf__use_idf=True, total=23.5min
[CV]  svc__C=5, tfidf__binary=True, tfidf__lowercase=True, tfidf__min_df=2, tfidf__ngram_range=(2, 4), tfidf__norm=l2, tfidf__stop_words=None, tfidf__use_idf=True, total=22.6min
[CV]  svc__C=5, tfidf__binary=True, tfidf__lowercase=True, tfidf__min_df=2, tfidf__ngram_range=(1, 4), tfidf__norm=l2, tfidf__stop_words=None, tfidf__use_idf=True, total=24.8min
[CV]  svc__C=5, tfidf__binary=True, tfidf__lowercase=True, tfidf__min_df=1, tfidf__ngram_range=(1, 4), tfidf__norm=l2, tfidf__stop_words=None, tfidf__use_idf=True, total=34.2min
[CV]  svc__C=5, tfidf__binary=True, tfidf__lowercase=True, tfidf__min_df=2, tfidf__ngram_range=(2, 4), tfidf__

[CV]  svc__C=5, tfidf__binary=True, tfidf__lowercase=True, tfidf__min_df=1, tfidf__ngram_range=(2, 4), tfidf__norm=l2, tfidf__stop_words=None, tfidf__use_idf=True, total=32.6min
[CV]  svc__C=5, tfidf__binary=True, tfidf__lowercase=True, tfidf__min_df=3, tfidf__ngram_range=(1, 4), tfidf__norm=l2, tfidf__stop_words=None, tfidf__use_idf=True, total=21.1min
[CV]  svc__C=5, tfidf__binary=True, tfidf__lowercase=True, tfidf__min_df=3, tfidf__ngram_range=(1, 4), tfidf__norm=l2, tfidf__stop_words=None, tfidf__use_idf=True, total=21.5min
[CV]  svc__C=5, tfidf__binary=True, tfidf__lowercase=True, tfidf__min_df=3, tfidf__ngram_range=(2, 4), tfidf__norm=l2, tfidf__stop_words=None, tfidf__use_idf=True, total=16.0min
[CV]  svc__C=5, tfidf__binary=True, tfidf__lowercase=True, tfidf__min_df=3, tfidf__ngram_range=(2, 4), tfidf__norm=l2, tfidf__stop_words=None, tfidf__use_idf=True, total=16.2min
[CV]  svc__C=5, tfidf__binary=True, tfidf__lowercase=True, tfidf__min_df=3, tfidf__ngram_range=(2, 4), tfidf__

[Parallel(n_jobs=-1)]: Done 600 out of 600 | elapsed: 120.8min finished
/home/unicph.domain/xwf576/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_search.py:706: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dtype=np.int)


saving model


['results/grid_search_object_masked_not_criminal.pkl']

#### Load data

In [27]:
criminal = joblib.load('results/grid_search_object_masked_criminal.pkl')
criminal_ids_test = pd.read_csv("document_ids_criminal_test",header=None)[1]
criminal_ids_train = pd.read_csv("document_ids_criminal_train",header=None)[1]
criminal_train_X = df_criminal.loc[df_criminal["id_verdict"].isin(criminal_ids_train), "verdict_text_masked"] 
criminal_test_X = df_criminal.loc[df_criminal["id_verdict"].isin(criminal_ids_test), "verdict_text_masked"] 
criminal_test_Y = df_criminal.loc[df_criminal["id_verdict"].isin(criminal_ids_test), "LABEL_party_who_pays"]
criminal_test_Y = Encoder.fit_transform(criminal_test_Y)

In [28]:
not_criminal = joblib.load('results/grid_search_object_masked_not_criminal.pkl')
not_criminal_ids_test = pd.read_csv("document_ids_not_criminal_test",header=None)[1]
not_criminal_ids_train = pd.read_csv("document_ids_not_criminal_train",header=None)[1]
not_criminal_train_X = df_not_criminal.loc[df_not_criminal["id_verdict"].isin(not_criminal_ids_train), "verdict_text_masked"] 
not_criminal_test_X = df_not_criminal.loc[df_not_criminal["id_verdict"].isin(not_criminal_ids_test), "verdict_text_masked"] 
not_criminal_test_Y = df_not_criminal.loc[df_not_criminal["id_verdict"].isin(not_criminal_ids_test), "LABEL_party_who_pays"]
not_criminal_test_Y = Encoder.fit_transform(not_criminal_test_Y)

### Metrics for each classifier

#### Predictions for criminal cases

In [29]:
# Classifier - Algorithm - SVM
# fit the training dataset on the classifier
# predict the labels on validation dataset
predictions_SVM_criminal = criminal.predict(criminal_test_X)
# Use accuracy_score function to get the accuracy
print_scores(predictions_SVM_criminal,criminal_test_Y)

SVM Accuracy Score -> 74.55516014234875
F1 Score -> 74.23134268537073
Confusion matrix: 
 [[241  77]
 [ 66 178]]


/home/unicph.domain/xwf576/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/base.py:326: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  indices = (scores > 0).astype(np.int)


In [41]:
criminal.best_estimator_.steps

[('tfidf', TfidfVectorizer(analyzer='word', binary=True, decode_error='strict',
          dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
          lowercase=True, max_df=1.0, max_features=None, min_df=3,
          ngram_range=(1, 4), norm='l2', preprocessor=None, smooth_idf=True,
          stop_words=['og', 'i', 'jeg', 'det', 'at', 'en', 'den', 'til', 'er', 'som', 'på', 'de', 'med', 'han', 'af', 'for', 'ikke', 'der', 'var', 'mig', 'sig', 'men', 'et', 'har', 'om', 'vi', 'min', 'havde', 'ham', 'hun', 'nu', 'over', 'da', 'fra', 'du', 'ud', 'sin', 'dem', 'os', 'op', 'man', 'hans', 'hvor',..., 'din', 'nogle', 'hos', 'blive', 'mange', 'ad', 'bliver', 'hendes', 'været', 'thi', 'jer', 'sådan'],
          strip_accents=None, sublinear_tf=False,
          token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
          vocabulary=None)),
 ('svc', LinearSVC(C=5, class_weight=None, dual=True, fit_intercept=True,
       intercept_scaling=1, loss='squared_hinge', max_iter=

In [37]:
dummy_classifier = dummy.DummyClassifier()
Tfidf_vect = TfidfVectorizer(analyzer='word')
Tfidf_vect.fit(df_criminal["verdict_text_masked"])
Train_X_Tfidf = Tfidf_vect.transform(criminal_train_X)
Test_X_Tfidf = Tfidf_vect.transform(criminal_test_X)
dummy_classifier.fit(Test_X_Tfidf,criminal_test_Y)
predictions_DUMMY = dummy_classifier.predict(Test_X_Tfidf)
print_scores(predictions_DUMMY,criminal_test_Y)

SVM Accuracy Score -> 50.355871886120994
F1 Score -> 49.83988867384315
Confusion matrix: 
 [[170 142]
 [137 113]]


/home/unicph.domain/xwf576/anaconda3/lib/python3.7/site-packages/sklearn/dummy.py:227: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  k in range(self.n_outputs_)).T


#### Predictions for not criminal cases

In [32]:
# Classifier - Algorithm - SVM
# fit the training dataset on the classifier
# predict the labels on validation dataset
predictions_SVM_not_criminal = not_criminal.predict(not_criminal_test_X)
# Use accuracy_score function to get the accuracy
print_scores(predictions_SVM_not_criminal,not_criminal_test_Y)


SVM Accuracy Score -> 70.89923181201988
F1 Score -> 63.70813031299307
Confusion matrix: 
 [[ 378  109   93]
 [  41  162   39]
 [ 149  213 1029]]


In [42]:
not_criminal.best_estimator_.steps

[('tfidf', TfidfVectorizer(analyzer='word', binary=True, decode_error='strict',
          dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
          lowercase=True, max_df=1.0, max_features=None, min_df=3,
          ngram_range=(2, 4), norm='l2', preprocessor=None, smooth_idf=True,
          stop_words=['og', 'i', 'jeg', 'det', 'at', 'en', 'den', 'til', 'er', 'som', 'på', 'de', 'med', 'han', 'af', 'for', 'ikke', 'der', 'var', 'mig', 'sig', 'men', 'et', 'har', 'om', 'vi', 'min', 'havde', 'ham', 'hun', 'nu', 'over', 'da', 'fra', 'du', 'ud', 'sin', 'dem', 'os', 'op', 'man', 'hans', 'hvor',..., 'din', 'nogle', 'hos', 'blive', 'mange', 'ad', 'bliver', 'hendes', 'været', 'thi', 'jer', 'sådan'],
          strip_accents=None, sublinear_tf=False,
          token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
          vocabulary=None)),
 ('svc', LinearSVC(C=5, class_weight=None, dual=True, fit_intercept=True,
       intercept_scaling=1, loss='squared_hinge', max_iter=

In [34]:
dummy_classifier = dummy.DummyClassifier()
Tfidf_vect = TfidfVectorizer(analyzer='word')
Tfidf_vect.fit(df_not_criminal["verdict_text_masked"])
Train_X_Tfidf = Tfidf_vect.transform(not_criminal_train_X)
Test_X_Tfidf = Tfidf_vect.transform(not_criminal_test_X)
dummy_classifier.fit(Test_X_Tfidf,not_criminal_test_Y)
predictions_DUMMY = dummy_classifier.predict(Test_X_Tfidf)
print_scores(predictions_DUMMY,not_criminal_test_Y)

SVM Accuracy Score -> 39.62946226841392
F1 Score -> 34.18899907680573
Confusion matrix: 
 [[156 131 292]
 [116 108 256]
 [296 245 613]]


/home/unicph.domain/xwf576/anaconda3/lib/python3.7/site-packages/sklearn/dummy.py:227: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  k in range(self.n_outputs_)).T


### Evaluate weights

#### Print the absolute largest coefficient for the criminal case classifier:

In [35]:
tf_idf = criminal.best_estimator_.get_params()["steps"][0][1]
tf_idf.fit(df_criminal["verdict_text_masked"])
svc = criminal.best_estimator_.get_params()["steps"][1][1]
tf_idf_X_Train = tf_idf.transform(criminal_train_X)
svc.fit(tf_idf_X_Train,Train_Y)
print_top10(tf_idf,svc,svc.classes_)

ValueError: Found input variables with inconsistent numbers of samples: [2246, 8850]